In [ ]:
```{bash}
# Set the year range of preference
years=("2020" "2021" "2022")

# Create a temporary directory to prevent server from crashing due to large file size (alternative is to create a home directory with mkdir)
temp_dir=$(mktemp -d)
   echo "Downloading to temporary directory: $temp_dir"

# Create loop to run download files for each of the years
for i in ${years[@]}; do
    # Download the .gz file into the temp directory
    curl -s -o "${temp_dir}/${i}.csv.gz" https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/${i}.csv.gz  
    
    # Decompress the file in the temp directory
    gunzip "${temp_dir}/${i}.csv.gz"  
    
    # Count the lines in the decompressed CSV file
    count=$(wc -l < "${temp_dir}/${i}.csv")
    echo "Year ${i} has ${count} observations.";
    
done
```

In [ ]:
```{bash}
# Obtain the station code of preference
DVcode=$(curl -s https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt | grep -i "Death Valley" ghcnd-stations.txt | awk '{print $1}')
echo "Death Valley code is set to: ${DVcode}"

# Set up output file for combined dataset
output="DeathValley_comb.csv"
> "$output" # Clear out file contents if the file exists previously 

# Loop through each yearly datafile to extract information for station of choice
echo "File is being processed..."
for i in ${years[@]}; do \
    sed -n -e "/^${DVcode}/p" "${temp_dir}/${i}.csv" | \
    grep "TMAX" | \
    grep -E '^[^,]*,[0-9]{4}03[0-9]{2},' >> "$output"; \ 
done
echo "Processing complete."
```

In [ ]:
 ```{python}
station_file = "Downloads/DeathValley_comb.csv"  # Update the path if 'output' is in another directory

try:
    # Open the file and read its contents
    with open(station_file, 'r') as file:
        # Read and process all lines from the file
        lines = file.readlines()

        for line in lines:
            line = line.strip()  # Remove leading/trailing whitespace and newlines
            
except FileNotFoundError:
    print(f"Error: The file '{station_file}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")
 ```

In [ ]:
 ```{python}
import pandas as pd
import matplotlib.pyplot as plt
try:
    # Read the CSV file into a Pandas DataFrame
    df = pd.read_csv(station_file, header=None)

    # Rename the columns for easier reference
    df.columns = ['StationID', 'Date', 'Category', 'Value', 'Col5', 'Col6', 'Col7', 'Col8']

    # Convert the 'Date' column to datetime format
    df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')

    # Filter the data to only include entries from March
    df_marchdata = df[df['Date'].dt.month == 3]

    # Group the data by day of March (extracted from 'Date')
    df_marchdata['Day'] = df_marchdata['Date'].dt.day

    # Create a list of 'Value' data, grouped by 'Day'
    values_by_day = [group['Value'].values for name, group in df_marchdata.groupby('Day')]

    # Create the box plot with days as the x-axis labels
    plt.figure(figsize=(10, 6))
    plt.boxplot(values_by_day, tick_labels=df_marchdata['Day'].unique())

    # Customize the plot
    plt.title('Box Plot of Temperature Values in March')
    plt.xlabel('Day')
    plt.ylabel('Temperature (TMAX)')
    plt.grid(True)

    # Show the plot
    plt.show()


except FileNotFoundError:
    print(f"Error: The file '{station_file}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")
 ```

In [1]:
```{bash}
get_weather() {
    # Check if user requests help
    if [ "$1" == "-h" ]; then
        echo "Usage: get_weather LOCATION TYPE YEARS MONTH"
        echo "Example: get_weather 'Death Valley' TMAX '2010 2011' 03"
        echo "Arguments:"
        echo "  LOCATION - A string to identify the location (e.g., Death Valley)"
        echo "  TYPE - Weather variable of interest (e.g., TMAX, TMIN)"
        echo "  YEARS    - Years of interest (e.g., '2010 2011 2012')"
        echo "  MONTH    - Month of interest in two-digit format (e.g., 03 for March)"
        return 0
    fi

    # Check for the correct number of arguments
    if [ "$#" -lt 4 ]; then
        echo "Error: Incorrect number of arguments, you must provide at least 4 arguments."
        echo "Usage: get_weather LOCATION TYPE YEARS MONTH"
        echo "Example: get_weather 'Death Valley' TMAX '2010 2011' 03"
        return 1
    fi

    LOCATION="$1"
    TYPE="$2"
    YEARS="$3"
    MONTH="$4"

    # Create a temporary directory
    temp_dir2=$(mktemp -d)
    echo "Temporary directory created at: $temp_dir2"

    # Download the station data file and find the location code
    station_file="${temp_dir}/ghcnd-stations.txt"
    curl -s https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt -o "$station_file"
    
    # Search for the location string in the station file
    LOCATION_CODE=$(grep -i "$LOCATION" "$station_file" | awk '{print $1}')

    # Check if the location code was found
    if [ -z "$LOCATION_CODE" ]; then
        echo "Error: No weather station found for location: $LOCATION"
        rm -rf "$temp_dir2"  # Clean up
        return 1
    fi

    echo "Location code for $LOCATION: $LOCATION_CODE"

    # Loop through the years and download data for each year
    for year in $YEARS; do
        data_file="${temp_dir2}/${year}.csv.gz"
        csv_file="${temp_dir2}/${year}.csv"

        # Download and decompress the data for the given year
        echo "Downloading data for $year..."
        curl -s "https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/${year}.csv.gz" -o "$data_file"
        gunzip "$data_file"

        # Extract relevant data (location code, month, and weather variable)
        echo "Processing data for $year, $MONTH..."
        awk -F, -v code="$LOCATION_CODE" -v var="$TYPE" -v month="$MONTH" '
         $1 == code && substr($2, 5, 2) == month && $3 == var { print $0 }
        ' "$csv_file"
    done

    # Clean up the temporary directory
    echo "Cleaning up temporary files..."
    rm -rf "$temp_dir2"

    echo "Process completed."
}
```

SyntaxError: unterminated string literal (detected at line 60) (4015783920.py, line 60)